In [1]:
import boto3
import time
import zipfile
from io import BytesIO
import sagemaker

In [9]:
# 630 MB dataset
#DATASET_SOURCE = {
#    'Bucket': 'aws-jam-challenge-resources',
#    'Key': 'covid-19-with-sage-maker/radiography_train_data.zip'
#}

# 3 MB dataset
#DATASET_SOURCE = {
#    'Bucket': 'aws-jam-challenge-resources',
#    'Key': 'covid-19-with-sage-maker/radiography_train_data_10.zip'
#}


# 630 MB dataset
#LOCAL_ZIP = 'dataset/radiography_train_data.zip'

# 3 MB dataset
LOCAL_ZIP = 'dataset/radiography_train_data_10.zip'

### Fetching default S3 Bucket in the JAM Account

In [5]:
s3_resource = boto3.resource('s3')

all_buckets = s3_resource.buckets.all()
for bucket in all_buckets:
    if (bucket.name.startswith('sagemaker')):
        BUCKET = bucket.name
        print ("SageMaker Default Bucket for Training Data: ", BUCKET)
        
jam_bucket = s3_resource.Bucket(BUCKET)

SageMaker Default Bucket for Training Data:  sagemaker-us-east-1-445757850280


### Creating S3 Bucket in the JAM Account

In [7]:
# Creating Default S3 Bucket in the JAM Account
'''
# get the region and sagemaker session
region = boto3.Session().region_name
session = sagemaker.Session()

# create a sagemaker default bucket
BUCKET = session.default_bucket() 
print ("Created Default Bucket: ", BUCKET)

s3_resource = boto3.resource('s3')
jam_bucket = s3_resource.Bucket(BUCKET)
'''

# Creating Default S3 Bucket in the JAM Account
'''
BUCKET = "covid-19-image-dataset-"+str(int(time.time()))
REGION = "us-west-1"
s3_client = boto3.client('s3', region_name=REGION)
s3_resource = boto3.resource('s3')
jam_bucket = s3_resource.Bucket(BUCKET)

print ("Creating Bucket: ", BUCKET)
s3_client.create_bucket(Bucket=BUCKET,
                        CreateBucketConfiguration={'LocationConstraint': REGION})
'''

'\nBUCKET = "covid-19-image-dataset-"+str(int(time.time()))\nREGION = "us-west-1"\ns3_client = boto3.client(\'s3\', region_name=REGION)\ns3_resource = boto3.resource(\'s3\')\njam_bucket = s3_resource.Bucket(BUCKET)\n\nprint ("Creating Bucket: ", BUCKET)\ns3_client.create_bucket(Bucket=BUCKET,\n                        CreateBucketConfiguration={\'LocationConstraint\': REGION})\n'

### Copying zipped Dataset into the local S3 Bucket

In [8]:
'''
print ("Copying Data from: ", DATASET_SOURCE)
jam_bucket.copy(DATASET_SOURCE, LOCAL_ZIP)
'''

'\nprint ("Copying Data from: ", DATASET_SOURCE)\njam_bucket.copy(DATASET_SOURCE, LOCAL_ZIP)\n'

### Unzipping Dataset

In [10]:
print ("Unzipping ", LOCAL_ZIP)
zip_obj = s3_resource.Object(bucket_name=BUCKET, key=LOCAL_ZIP)
buffer = BytesIO(zip_obj.get()["Body"].read())
z = zipfile.ZipFile(buffer)
for filename in z.namelist():
    file_info = z.getinfo(filename)
    s3_resource.meta.client.upload_fileobj(
        z.open(filename),
        Bucket=BUCKET,
        Key=f'{filename}'
    )
print ("Completed Unzipping Training Data")
jam_bucket.objects.filter(Prefix="dataset/").delete()

Unzipping  dataset/radiography_train_data_10.zip
Completed Unzipping Training Data


[{'ResponseMetadata': {'RequestId': '6G010HD553H6FCHP',
   'HostId': 'miRDf9VfUw/0kvLXVh0WvvWzD9OO7gEX2IGZUmaIJZl+qY/gEkyNeLTui6JfU+HTmXrc329b1KQ=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'miRDf9VfUw/0kvLXVh0WvvWzD9OO7gEX2IGZUmaIJZl+qY/gEkyNeLTui6JfU+HTmXrc329b1KQ=',
    'x-amz-request-id': '6G010HD553H6FCHP',
    'date': 'Wed, 03 Nov 2021 11:28:40 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'dataset/radiography_train_data_10.zip'}]}]

## Compute average file size

In [13]:
def get_size(bucket, path):
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket)
    total_size = 0
    image_count = 0
    for obj in my_bucket.objects.filter(Prefix=path):
        if ".png" in obj.key:
            total_size = total_size + obj.size
            image_count = image_count + 1

    return total_size, image_count

folder_size, image_count = get_size(BUCKET, 'radiography_train_data_10')
average_image_size = folder_size / image_count
print (average_image_size)

37378.775
